# Example 1 - The farmer's problem

Recall the farmers problems, who is trying to determine what is the best land allocation for her crops given the uncertainty in the production yield.


In [16]:
# Loading the packages we need
using JuMP 
using Gurobi

The deterministic data for the problem is as below:

In [17]:
max_area = 500.0;
crops = [:wheat, :corn, :sugar_beet];
plant_cost = Dict(
    :wheat      => 150.0,
    :corn       => 230.0,
    :sugar_beet => 260.0
);
min_qt = Dict(
    :wheat      => 200.0,
    :corn       => 240.0,
    :sugar_beet => 0.0
);

quota_max = Dict(
    :wheat      => 200000, # This is large enough to not limit production
    :corn       => 200000, 
    :sugar_beet => 6000.0
);

sell_quota = Dict(
    :wheat      => 170.0,
    :corn       => 150.0,
    :sugar_beet => 36.0
);

sell_above = Dict(
    :wheat      => 0.0,
    :corn       => 0.0,
    :sugar_beet => 10.0
);

buy_price = Dict(
    :wheat      => 238.0,
    :corn       => 210.0,
    :sugar_beet => 1000.0
);

mean_yield = Dict(
    :wheat      => 2.5,
    :corn       => 3.0,
    :sugar_beet => 20.0
);

The deterministic model has the following formulation:

Let $I=\{1:\text{wheat}, 2:\text{corn}, 3:\text{sugar beets}\}$. Then
- $x_i$	- acres devoted to $i$
- $y_i$ - tons of $i$ purchased, $i \in I \setminus \{3\}$
- $w_i$ - tons of $i$ sold, $i \in I \cup \{4\}$, $\{4: \text{sugar beets (over quota)}\}$.
	
The farmer's problem is:

\begin{align*}
    \min & 150x_1 + 230 x_2 + 260 x_3 + 238 y_1 + 210 y_2 \\ & - 170 w_1 - 150 w_2 - 36 w_3 - 10w_4 \\
    \text{s.t.:~} & x_1 + x_2 + x_3 \le 500 \\
    & 2.5x_1 + y_1 - w_1 \ge 200 \\
    & 3 x_2 + y_2 - w_2 \ge 240 \\
    & w_3 + w_4 \le 20x_3 \\
    & w_3 \le 6000 \\
    & x_i \ge 0, i \in I; y_i \ge 0, i \in I \setminus \{3\}; w_i \ge 0, i \in I \cup \{4\}.
\end{align*}


In [18]:
fm_mean = Model(Gurobi.Optimizer)

@variable(fm_mean, x[c ∈ crops] ≥ 0)     # Planted area per crop
@variable(fm_mean, y[c ∈ crops] ≥ 0)     # Quantity sold below the quota
@variable(fm_mean, w[c ∈ crops] ≥ 0)     # Quantity sold above the quota (restricted by the max quota)
@variable(fm_mean, z[c ∈ crops] ≥ 0)     # Quantity bought

@constraint(fm_mean, sum(x[c] for c ∈ crops) ≤ max_area)   
@constraint(fm_mean, [c ∈ crops], x[c] * mean_yield[c] + z[c] - y[c] - w[c] ≥  min_qt[c])                                                            
@constraint(fm_mean, [c ∈ crops], y[c] ≤ quota_max[c])    

@expression(fm_mean, cost_det, sum(plant_cost[c] * x[c] for c ∈ crops))
@expression(fm_mean, cost_sto, sum((buy_price[c] * z[c] - sell_quota[c]*y[c] - sell_above[c] * w[c]) for c ∈ crops))       

@objective(fm_mean, Min, cost_det + cost_sto);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-03


We can check the mode to see if we got it right.

In [19]:
# Checking the model
print(fm_mean)

Min 150 x[wheat] + 230 x[corn] + 260 x[sugar_beet] + 238 z[wheat] - 170 y[wheat] + 210 z[corn] - 150 y[corn] + 1000 z[sugar_beet] - 36 y[sugar_beet] - 10 w[sugar_beet]
Subject to
 2.5 x[wheat] - y[wheat] - w[wheat] + z[wheat] ≥ 200.0
 3 x[corn] - y[corn] - w[corn] + z[corn] ≥ 240.0
 20 x[sugar_beet] - y[sugar_beet] - w[sugar_beet] + z[sugar_beet] ≥ 0.0
 x[wheat] + x[corn] + x[sugar_beet] ≤ 500.0
 y[wheat] ≤ 200000.0
 y[corn] ≤ 200000.0
 y[sugar_beet] ≤ 6000.0
 x[wheat] ≥ 0.0
 x[corn] ≥ 0.0
 x[sugar_beet] ≥ 0.0
 y[wheat] ≥ 0.0
 y[corn] ≥ 0.0
 y[sugar_beet] ≥ 0.0
 w[wheat] ≥ 0.0
 w[corn] ≥ 0.0
 w[sugar_beet] ≥ 0.0
 z[wheat] ≥ 0.0
 z[corn] ≥ 0.0
 z[sugar_beet] ≥ 0.0


We can then optimise the model

In [20]:
optimize!(fm_mean);

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 7 rows, 12 columns and 18 nonzeros
Model fingerprint: 0xec2ba12d
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 2e+05]
Presolve removed 3 rows and 2 columns
Presolve time: 0.00s
Presolved: 4 rows, 10 columns, 13 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -3.2000000e+32   1.000000e+30   3.200000e+02      0s
       3   -1.1860000e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds (0.00 work units)
Optimal objective -1.186000000e+05

User-callback calls 62, time in user-callback 0.00 sec


In [21]:
sol_plant_mean = round.(value.(x),digits=2)                 # Areas planted
sol_bought_mean = round.(value.(z),digits=2)                # Quantities bought
sol_sold_bq_mean = round.(value.(y),digits=2)               # Sold below quota
sol_sold_aq_mean = round.(value.(w),digits=2)               # Sold above quota
obj_mean = objective_value(fm_mean)

println("crops = ", crops)
println("sol_plant_mean = ",[sol_plant_mean[c] for c in crops])
println("sol_bought_mean = ",[sol_bought_mean[c] for c in crops])
println("sol_sold_bq_mean = ",[sol_sold_bq_mean[c] for c in crops])
println("sol_sold_aq_mean = ",[sol_sold_aq_mean[c] for c in crops])
println("obj = ", obj_mean)

crops = [:wheat, :corn, :sugar_beet]
sol_plant_mean = [120.0, 80.0, 300.0]
sol_bought_mean = [0.0, 0.0, 0.0]


sol_sold_bq_mean = [100.0, 0.0, 6000.0]
sol_sold_aq_mean = [0.0, 0.0, 0.0]
obj = -118600.0


Let us now generate the scenarios for the yields. We assume that all scenarios have the same probability 1/3 with `:good` being +20% above and `:bad` being 20% below the `:mean` scenario.  

In [22]:
scenarios = [:good, :mean, :bad];
yield_sto = Dict(
    :good => 1.2,
    :mean => 1,
    :bad  => 0.8
)

prob = Dict(
    :good => 1/3,
    :mean => 1/3,
    :bad  => 1/3
);

We can adapt the formulation accordingly. Notice the new set/ indices refering to the scenarios. Also, the variables are now indexed by `s in scenario` accordingly.

In [23]:
fm = Model(Gurobi.Optimizer)
@variable(fm, x[c ∈ crops] ≥ 0)                                              
@constraint(fm, sum(x[c] for c ∈ crops) ≤ max_area)                           
@expression(fm, first_cost, sum(plant_cost[c] * x[c] for c ∈ crops))             

@variable(fm, y[c ∈ crops, s ∈ scenarios] ≥ 0)                                
@variable(fm, w[c ∈ crops, s ∈ scenarios] ≥ 0)                                
@variable(fm, z[c ∈ crops, s ∈ scenarios] ≥ 0)                               

@constraint(fm, [c ∈ crops, s ∈ scenarios],                                   
    x[c] * (mean_yield[c] * yield_sto[s]) + z[c,s] - y[c,s] - w[c,s] ≥ min_qt[c]
)                                                                
@constraint(fm, [c ∈ crops,s ∈ scenarios], y[c,s] ≤ quota_max[c])                                                   

@expression(fm, second_cost[s ∈ scenarios],
            sum(buy_price[c] * z[c,s] - sell_quota[c] * y[c,s] - sell_above[c] * w[c,s] 
                for c ∈ crops)
)

@objective(fm, Min, first_cost + sum(prob[s] * second_cost[s] for s ∈ scenarios));


Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-03


In [24]:
# Checking the model
print(fm)

Min 150 x[wheat] + 230 x[corn] + 260 x[sugar_beet] + 79.33333333333333 z[wheat,good] - 56.666666666666664 y[wheat,good] + 70 z[corn,good] - 50 y[corn,good] + 333.3333333333333 z[sugar_beet,good] - 12 y[sugar_beet,good] - 3.333333333333333 w[sugar_beet,good] + 79.33333333333333 z[wheat,mean] - 56.666666666666664 y[wheat,mean] + 70 z[corn,mean] - 50 y[corn,mean] + 333.3333333333333 z[sugar_beet,mean] - 12 y[sugar_beet,mean] - 3.333333333333333 w[sugar_beet,mean] + 79.33333333333333 z[wheat,bad] - 56.666666666666664 y[wheat,bad] + 70 z[corn,bad] - 50 y[corn,bad] + 333.3333333333333 z[sugar_beet,bad] - 12 y[sugar_beet,bad] - 3.333333333333333 w[sugar_beet,bad]
Subject to
 3 x[wheat] - y[wheat,good] - w[wheat,good] + z[wheat,good] ≥ 200.0
 3.5999999999999996 x[corn] - y[corn,good] - w[corn,good] + z[corn,good] ≥ 240.0
 24 x[sugar_beet] - y[sugar_beet,good] - w[sugar_beet,good] + z[sugar_beet,good] ≥ 0.0
 2.5 x[wheat] - y[wheat,mean] - w[wheat,mean] + z[wheat,mean] ≥ 200.0
 3 x[corn] - y[cor

In [25]:
optimize!(fm);

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 19 rows, 30 columns and 48 nonzeros
Model fingerprint: 0x05721d1e
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 2e+05]
Presolve removed 9 rows and 6 columns
Presolve time: 0.00s
Presolved: 10 rows, 24 columns, 33 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -3.2000000e+32   3.000000e+30   3.200000e+02      0s
      14   -1.0839000e+05   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.00 seconds (0.00 work units)
Optimal objective -1.083900000e+05

User-callback calls 73, time in user-callback 0.00 sec


In [26]:
sol_plant = round.(value.(x),digits=2)          # Areas planted
sol_bought = round.(value.(z),digits=2)         # Quantities bought
sol_sold_bq = round.(value.(y),digits=2)        # Sold below quota
sol_sold_aq = round.(value.(w),digits=2);       # Sold above quota
obj = objective_value(fm)

println("sol_plant = ", [sol_plant[c] for c in crops])
println("obj = ", obj)
for s in scenarios
    println("scenario: ", s)
    println("   crops = ", crops)
    println("   sol_bought = ", [sol_bought[c,s] for c in crops])
    println("   sol_sold_bq = ", [sol_sold_bq[c,s] for c in crops])
    println("   sol_sold_aq = ",[sol_sold_aq[c,s] for c in crops])
end    

sol_plant = [170.0, 80.0, 250.0]
obj = -108390.0
scenario: good
   crops = [:wheat, :corn, :sugar_beet]
   sol_bought = [0.0, 0.0, 0.0]


   sol_sold_bq = [310.0, 48.0, 6000.0]
   sol_sold_aq = [0.0, 0.0, 0.0]
scenario: mean
   crops = [:wheat, :corn, :sugar_beet]
   sol_bought = [0.0, 0.0, 0.0]
   sol_sold_bq = [225.0, 0.0, 5000.0]
   sol_sold_aq = [0.0, 0.0, 0.0]
scenario: bad
   crops = [:wheat, :corn, :sugar_beet]
   sol_bought = [0.0, 48.0, 0.0]
   sol_sold_bq = [140.0, 0.0, 4000.0]
   sol_sold_aq = [0.0, 0.0, 0.0]


## Value of stochastic solution

To calculate the value of stochastic solution, we need to compare the 
1. performance between the solution we obtain considering only one scenario (`:mean` scenario in our case), 
$z^{\text{EV}} = \mathbb{E}_\xi\left[F(x(\overline{\xi}), \xi)\right],$ where $x(\overline{\xi}) = \argmin_x F(x, \overline{\xi})$ and $\overline{\xi}$ is the `:mean` scenario;
2. and that of the stochastic model.

For (1), we can simply add a constraint to the stochastic model that forces the first-stage solution to be the same of model with the average scenario only.

In [27]:
# Forms the EEV problem
@constraint(fm, [c in crops], x[c] == sol_plant_mean[c]);  
optimize!(fm);

println("EEV value = ", objective_value(fm))
println("VSS = ", obj - objective_value(fm))

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 22 rows, 30 columns and 51 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 2e+05]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.0839000e+05   2.500000e+02   0.000000e+00      0s
       1   -1.0724000e+05   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective -1.072400000e+05

User-callback calls 18, time in user-callback 0.00 sec
EEV value = -107240.0
VSS = -1150.0


## Expected Value of Perfect Information

To calculate the EVPI, we need to calculate the fully anticipative solution, meaninn that we need to calculate
\begin{equation*}
    z^{\text{WS}} = \mathbb{E}_\xi\left[ \min_x \{F(x,\xi)\} \right] = \mathbb{E}_\xi\left[F(x(\xi), \xi)\right],
\end{equation*}

This consists of calculating the optimal solution for each scenario, and then take the expected value.

Firest, let us solve again our stochastic model to refresh it to its original form:

In [28]:
# Restart the 2SSP model (C&P from above)
fm = Model(Gurobi.Optimizer)
set_silent(fm)
@variable(fm, x[c ∈ crops] ≥ 0)                                              
@constraint(fm, sum(x[c] for c ∈ crops) ≤ max_area)                           
@expression(fm, first_cost, sum(plant_cost[c] * x[c] for c ∈ crops))             

@variable(fm, y[c ∈ crops, s ∈ scenarios] ≥ 0)                                
@variable(fm, w[c ∈ crops, s ∈ scenarios] ≥ 0)                                
@variable(fm, z[c ∈ crops, s ∈ scenarios] ≥ 0)                               

@constraint(fm, [c ∈ crops, s ∈ scenarios],                                   
    x[c] * (mean_yield[c] * yield_sto[s]) + z[c,s] - y[c,s] - w[c,s] ≥ min_qt[c]
)                                                                
@constraint(fm, [c ∈ crops,s ∈ scenarios], y[c,s] ≤ quota_max[c])                                                   

@expression(fm, second_cost[s ∈ scenarios],
            sum(buy_price[c] * z[c,s] - sell_quota[c] * y[c,s] - sell_above[c] * w[c,s] 
                for c ∈ crops)
)

@objective(fm, Min, first_cost + sum(prob[s] * second_cost[s] for s ∈ scenarios));

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-03


In [29]:
optimize!(fm);
obj = objective_value(fm)

-108390.0

Our trick to calculate the WS solution is to use an auxiliary probability vector `aux_prob` that we can "mess with". 

That means, we first zero the dictionary containing the probability and then, for each scenario, we set that scenario to have probability 1. We then accumulate the value in the `WS` auxiliary variable, multiplied by the original probability.

In [30]:
# Calculating WS
aux_prob = copy(prob)
WS = 0.0

for (key, value) in aux_prob
    aux_prob[key] = 0
end

for (key, value) in prob
    aux_prob[key] = 1
    @objective(fm, Min, first_cost + sum(aux_prob[s] * second_cost[s] for s in scenarios))
    optimize!(fm)
    WS += prob[key] * objective_value(fm)
    aux_prob[key] = 0
end

println("The WS value is: ", WS)
println("EVPI is: ", WS - obj)

The WS value is: -115405.55555555555
EVPI is: -7015.5555555555475
